In [15]:
import cv2
import numpy as np

image = cv2.imread('./images/targets/img2.jpeg')

#image = cv2.resize(image, None, fx=0.5fy=0.5, interpolation=cv2.INTER_LANCZOS4)

imageOrig = image.copy()

bilateral = cv2.bilateralFilter(image,70,75,75)

bilateral = cv2.GaussianBlur(bilateral,(15,15),0)

median = cv2.medianBlur(image,5)
# cv2.imshow('',median)


kernelSharpen = np.array([[-2,-2,-2],
                          [-2,17,-2],
                          [-2,-2,-2]])

bilateral = cv2.filter2D(bilateral,-1,kernelSharpen)

cv2.imshow('',bilateral)


# cv2.imshow('',bilateral)
cv2.waitKey()

lower = np.array([87, 0, 102])
upper = np.array([179, 255, 255])

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv, lower, upper)

kernel1 = np.ones((5,5), np.uint8)

output = cv2.bitwise_and(image,image, mask= mask)


gray = cv2.cvtColor(output,cv2.COLOR_BGR2GRAY)

erosion = cv2.erode(gray,kernel1,iterations = 1)
dil = cv2.dilate(erosion,kernel1,iterations = 1)


edge = cv2.Canny(dil,50,255)
# cv2.imshow('',edge)
# cv2.waitKey()

# img3 = cv2.cvtColor(output,cv2.COLOR_BGR2GRAY)

contours, hierarchy = cv2.findContours(edge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    (x,y,w,h) = cv2.boundingRect(contour)
    croppedImage = edge[y:y+h,x:x+w] 

bg = np.zeros(gray.shape,dtype='uint8')
bg1 = np.zeros(gray.shape,dtype='uint8')
for c in contours:
    for i in range(len(c)):
        cont = c[i][0]
        bg1 = cv2.circle(bg1,(cont[0],cont[1]),27,75,1,8,0)
        bg = cv2.addWeighted(bg,1,bg1,0.5,0)
        bg1 = np.zeros(gray.shape,dtype='uint8')
k, bg = cv2.threshold(bg, 220, 255, cv2.THRESH_BINARY)
cv2.imshow('detected circles',bg)
cv2.waitKey(0)
print(bg)

ker = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
#accuracy = 0.01 * cv2.arcLength(contours[0], True)
#approx = cv2.approxPolyDP(contours[0], accuracy, True)
#cv2.drawContours(imageOrig, [approx], 0, (0,0,255),-1)
cv2.fillPoly(bg, [contours[0]], (255, 255, 255))
gray = cv2.medianBlur(bg, 15)
#circles = cv2.HoughCircles(edge,cv2.HOUGH_GRADIENT,1,10,
 #                           param1=50,param2=30,minRadius=0,maxRadius=0)
#circles = np.uint16(np.around(circles))
#for i in circles[0,:]:
    # draw the outer circle
#    cv2.circle(image,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
#    cv2.circle(image,(i[0],i[1]),2,(0,0,255),3)
#cv2.imshow('detected circles',image)
#cv2.waitKey(0)
#k, gray = cv2.threshold(gray, 100, 70, cv2.THRESH_BINARY_INV)
dist_transform = cv2.distanceTransform(gray,cv2.DIST_L2,3)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
cv2.imshow('ima', sure_fg)
cv2.waitKey()
# # Finds circles in a grayscale image using the Hough transform
# circles = cv2.HoughCircles(dil, cv2.HOUGH_GRADIENT, 1, 100,
#                              param1=100,param2=30,minRadius=0,maxRadius=200)

# # cv2.HoughCircles function has a lot of parameters, so you can find more about it in documentation
# # or you can use cv2.HoughCircles? in jupyter nootebook to get that 
# print(circles)
# # Check to see if there is any detection
# if circles is not None:
#     # If there are some detections, convert radius and x,y(center) coordinates to integer
#     circles = np.round(circles[0, :]).astype("int")

#     for (x, y, r) in circles:
#         # Draw the circle in the output image
#         cv2.circle(imageOrig, (x, y), r, (0,255,0), 3)
#         # Draw a rectangle(center) in the output image
#         cv2.rectangle(imageOrig, (x - 2, y - 2), (x + 2, y + 2), (0,255,0), -1)

# cv2.imshow("Detections",imageOrig)
# cv2.waitKey()

# cv2.imshow('',output)



# # find Harris corners
# gray = np.float32(croppedImage)
# dst = cv2.cornerHarris(gray,2,3,0.04)
# dst = cv2.dilate(dst,None)
# ret, dst = cv2.threshold(dst,0.01*dst.max(),255,0)
# dst = np.uint8(dst)
# # find centroids
# ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
# # define the criteria to stop and refine the corners
# criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
# corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)
# # Now draw them 
# res = np.hstack((centroids,corners))
# res = np.int0(res)
# # croppedImage[res[:,1],res[:,0]]=[0,0,255]
# # croppedImage[res[:,3],res[:,2]] = [0,255,0]
# # cv2.imwrite('subpixel5.png',img)
# cv2.imshow('',croppedImage)
# cv2.waitKey()
# cv2.imshow('',gray)
        

cv2.waitKey()
cv2.destroyAllWindows()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [3]:
import cv2
import numpy as np
import time

image = cv2.imread('./images/targets/img7.jpg')

imageOrig = image.copy()

bilateral = cv2.bilateralFilter(image,70,75,75)

bilateral = cv2.GaussianBlur(bilateral,(15,15),0)

kernelSharpen = np.array([[-2,-2,-2],
                          [-2,17,-2],
                          [-2,-2,-2]])

bilateral = cv2.filter2D(bilateral,-1,kernelSharpen)

cv2.imshow('',bilateral)
cv2.waitKey()



lower = np.array([87, 0, 102])
upper = np.array([179, 255, 255])

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv, lower, upper)

kernel1 = np.ones((5,5), np.uint8)
output = cv2.bitwise_and(image,image, mask= mask)

gray = cv2.cvtColor(output,cv2.COLOR_BGR2GRAY)

erosion = cv2.erode(gray,kernel1,iterations = 1)
dil = cv2.dilate(erosion,kernel1,iterations = 1)
edge = cv2.Canny(dil,20,255)
# cv2.imshow('',edge)
# cv2.waitKey()

# img3 = cv2.cvtColor(output,cv2.COLOR_BGR2GRAY)
print(edge)
contours, hierarchy = cv2.findContours(edge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    (x,y,w,h) = cv2.boundingRect(contour)
    croppedImage = edge[y:y+h,x:x+w] 

bg = np.zeros(gray.shape,dtype='uint8')


for contour in contours:
    accuracy = 0.009* cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, accuracy, True)
    cv2.drawContours(imageOrig, [approx], 0, (0,0,255),2)
    cv2.fillPoly(bg, [approx], (255, 255, 255), 8)

gray = cv2.medianBlur(bg, 15)
edge = cv2.Canny(gray,20,255)
cv2.imshow('4',edge)
cv2.waitKey()
rows = gray.shape[0]
circles = cv2.HoughCircles(edge, cv2.HOUGH_GRADIENT, 1, 1, param1=50, param2=30, minRadius=0, maxRadius=0)

contours, hierarchy = cv2.findContours(edge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

(x,y,w,h) = cv2.boundingRect(contours[0])
r = 15
d = r*2
arr = [] 
for i in range(y,(y+h)-d, 1):
    for j in range(x,(x+w)-r, 1):
        crop = bg[i:i+d,j:j+d]
        msk = np.ones((d,d), np.uint8)
        cv2.circle(msk,(r, r), r, (255,0,0), -1)
        cir = cv2.bitwise_and(crop,msk)
        n = np.sum(cir == 255)
        arr.append((j+r,i+r, n))

def BiggestCrc(crc):
    return crc[2]

num = 5
arr.sort(reverse=True, key=BiggestCrc)

# time.sleep(200)
ma = 0
for a in arr:
    if ma < a[2]:
        ma = a[2]
print(x)
print(y)
print(x+w)
print(y+h)
bound = 100
for a in range(len(arr)):
    if arr[a][2] >= ma:
        if arr[a][0] < x+bound or arr[a][0] > x+w-bound:
            print(arr[a])
            if arr[a][1] < y+bound or arr[a][1] > y+h-bound:
                cv2.circle(image, (arr[a][0],arr[a][1]), r, (0,0,255), 1)
                cv2.rectangle(image,(x+bound,y+bound), (x+w-bound,y+h-bound), (255,0,0),2)
                cv2.imshow('1',image)
    
    
cv2.waitKey()
cv2.destroyAllWindows()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
170
348
197
378
